In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import sklearn

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Завантажуємо потрібні дані

In [ ]:
train = pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv')
test = pd.read_csv('/kaggle/input/home-credit-default-risk/application_test.csv')
descr = pd.read_csv('/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv', encoding='latin1')

---

# Перший погляд на таблиці

У файлі `HomeCredit_columns_description.csv` знаходяться описи кожної колонки з датасету

In [ ]:
descr[descr['Table'] == 'application_{train|test}.csv'][['Row', 'Description']]

Як можна побачити (та прочитати з опису даних на [https://www.kaggle.com/c/home-credit-default-risk/data](https://www.kaggle.com/c/home-credit-default-risk/data)), кожен рядок даних - це дані клієнта, що хоче взяти позику, причому в трейновому датасеті є колонка `TARGET`, 1 в якій означає, що у цієї людини складнощі з оплатою. Інші колонки в обох датасетах - це різна інформація про людину: вік, стать, наявність авто/нерухомості/квартири/дітей, дохід, наявність якихось документів тощо. 

Перші декілька рядків обох таблиць

In [ ]:
print('train data shape: ', train.shape)
train.head()

Трейновий датасет містить 307511 прикладів

Типи даних в трейновому датасеті:

In [ ]:
train.dtypes.value_counts()

In [ ]:
print('test data shape: ', test.shape)
test.head()

Тестовий датасет містить 48744 прикладів

Типи даних в тестовому датасеті:

In [ ]:
test.dtypes.value_counts()

Колонок з типом `int64` на одну менше через відсутність колонки `TARGET`. Тип даних `object` - це колонки, значеннями яких є `string`'и

Подивимося на розподіл значень в колонці `TARGET` в трейновому датасеті

In [ ]:
negative_count = train['TARGET'].value_counts()[0]
positive_count = train['TARGET'].value_counts()[1]
print(f"don\'t have problems: {negative_count}, {negative_count/len(train):.3f}%")
print(f"have problems: {positive_count}, {positive_count/len(train):.3f}%")
train['TARGET'].plot.hist()

Бачимо, що значень 0 там значно більше, ніж 1, тобто клієнтів з фінансовими проблемами в трейновому датасеті значно менше

---

# Пропущені значення

Дослідимо наявність, кількість та частки пропущених значень у колонках обох датасетів

In [ ]:
def count_missing(df):
    mis_val = df.isnull().sum()
    mis_percent = 100*mis_val/(len(df))
    result_table = pd.DataFrame(
        {
            "Missing values": mis_val,
            "% of all values": mis_percent
        }).sort_values("% of all values", ascending=False).round(1)
    return result_table

In [ ]:
mis_val_table_train = count_missing(train)
mis_val_table_train[mis_val_table_train["% of all values"] > 0]

In [ ]:
columns_with_lots_null_train = len(mis_val_table_train[mis_val_table_train["% of all values"] > 50])
print(f'number of columns with >50% missing values: {columns_with_lots_null_train}')
print(f'{columns_with_lots_null_train/len(train.columns):.3f}% of columns have >50% missing values')
mis_val_table_train["% of all values"].hist()

Як видно з гістограми, більше половини колонок не мають або майже не мають пропущенних значень, але 33% колонок мають більше половини пропущених значень

Зробимо таке ж дослідження для тестового датасету

In [ ]:
mis_val_table_test = count_missing(test)
mis_val_table_test[mis_val_table_test["% of all values"] > 0]

In [ ]:
columns_with_lots_null_test = len(mis_val_table_test[mis_val_table_test["% of all values"] > 50])
print(f'number of columns with >50% missing values: {columns_with_lots_null_test}')
print(f'{columns_with_lots_null_test/len(test.columns):.3f}% of columns have >50% missing values')
mis_val_table_test["% of all values"].hist()

На тестовому датасеті ситуація приблизно така ж, як і на трейновому, причому колонки з найбільшою кількістю пропущених значень ті ж самі

Пропущенні значення при навчанні моделі заповнимо медіанними по кожній колонці

---

# Категорійні змінні

Як було виявлено вище, в обох датасетах є 16 колонок типу `object`, яким відповідають категорійні змінні. Подивимося на кількість унікальних значень кожної.

Трейовний датасет:

In [ ]:
unique_categories_train = train.select_dtypes('object').apply(pd.Series.unique, axis=0)

for c in zip(unique_categories_train.index, unique_categories_train):
    print(f'{c[0]} has {len(c[1])} unique values:\n{c[1]}\n')

Тестовий датасет:

In [ ]:
unique_categories_test = test.select_dtypes('object').apply(pd.Series.unique, axis=0)

for c in zip(unique_categories_test.index, unique_categories_test):
    print(f'{c[0]} has {len(c[1])} unique values:\n{c[1]}\n')

Логічно закодувати змінні, що мають 2 унікальних значення, за допомогою 0/1, а інші - за допомогою one-hot кодування

In [ ]:
# one-hot
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [ ]:
print('train data shape: ', train.shape)
print('test data shape: ', test.shape)

Кодування категорійних змінних збільшило кількість колонок вдвічі, але через те, що деяких значень могло не бути в якомусь з датасетів, кількість колонок відрізняється (з урахуванням колонки `TARGET`), це треба виправити

In [ ]:
train_labels = train['TARGET']
train, test = train.align(test, join='inner', axis=1)
print('train data shape: ', train.shape)
print('test data shape: ', test.shape)

---

# Аномалії в даних

Аномалії можуть траплятися через помилки при введені даних чи їх вимірюванні. Подивимося на колонки `DAYS_BIRTH` та `DAYS_EMPLOYED`, що показують вік людей у днях та кількість днів, які людина працює на поточній роботі на момент подачі заяви, оскільки в таких даних природно очікувати помилок при введенні.

In [ ]:
print('DAYS_BIRTH:',
      descr[descr['Row'] == 'DAYS_BIRTH']['Description'].to_numpy()[0])
print('DAYS_EMPLOYED:',
      descr[descr['Row'] == 'DAYS_EMPLOYED']['Description'].to_numpy()[0])

Трейновий датасет

In [ ]:
print((train['DAYS_BIRTH'] / -365).describe())
plt.boxplot(train['DAYS_BIRTH'] / -365, vert=False)
plt.show()

In [ ]:
print((train['DAYS_EMPLOYED'] / -365).describe())
plt.boxplot((train['DAYS_EMPLOYED'] / -365), vert=False)
plt.show()

In [ ]:
print(len(train[train['DAYS_EMPLOYED'] == 365243]))

Йой, в `train['DAYS_EMPLOYED']` є аж 55 тисяч людей, що працюють -1000 років

In [ ]:
anom_negative = sum(train_labels[train[train['DAYS_EMPLOYED'] == 365243].index.values] == 0)
anom_positive = sum(train_labels[train[train['DAYS_EMPLOYED'] == 365243].index.values] == 1)
not_anom_negative = sum(train_labels[train[train['DAYS_EMPLOYED'] != 365243].index.values] == 0)
not_anom_positive = sum(train_labels[train[train['DAYS_EMPLOYED'] != 365243].index.values] == 1)
anom_count = len(train[train['DAYS_EMPLOYED'] == 365243])
not_anom_count = len(train) - anom_count
print("anomalies:")
print(f"    don\'t have problems: {anom_negative/anom_count:.3f}%")
print(f"    have problems: {anom_positive/anom_count:.3f}%")
print("other:")
print(f"    don\'t have problems: {not_anom_negative/not_anom_count:.3f}%")
print(f"    have problems: {not_anom_positive/not_anom_count:.3f}%")

Цікаво, що частка людей без проблем з оплатою серед "аномальних" трохи більше, ніж серед "звичайних". Не дарма вони працюють тисячу років, не дарма...

Замінимо значення `DAYS_EMPLOYED` у цих робітників тисячоліття (та, якщо такі є, в тестовому датасеті теж) на `nan`, а потім вже при тренуванні моделі заповнимо їх медіанними. Також, додамо колонку, що вказуватиме, чи було це значення аномальним.

In [ ]:
train['DAYS_EMPLOYED_ANOM'] = 1*(train['DAYS_EMPLOYED'] == 365243)
test['DAYS_EMPLOYED_ANOM'] = 1*(test['DAYS_EMPLOYED'] == 365243)
train["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)
test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)

In [ ]:
print((train['DAYS_EMPLOYED'] / -365).describe())
plt.boxplot((train['DAYS_EMPLOYED'] / -365).dropna(), vert=False)
plt.show()

Тепер розподіл років роботи більш прийнятний, але треба перевірити, чи дійсно окремі люди працюють по 40-50 років (чи не виявиться їх вік менше)

In [ ]:
print(len(train[train['DAYS_EMPLOYED']/(-365) >= train['DAYS_BIRTH']/(-365)]))
print(len(test[test['DAYS_EMPLOYED']/(-365) >= test['DAYS_BIRTH']/(-365)]))

Таких випадків немає, тому тепер залишаємо як є, подивимося ще на такі ж розподіли на тестовому датасеті

In [ ]:
print((test['DAYS_BIRTH'] / -365).describe())
plt.boxplot(test['DAYS_BIRTH'] / -365, vert=False)
plt.show()

In [ ]:
print((test['DAYS_EMPLOYED']/-365).describe())
plt.boxplot((test['DAYS_EMPLOYED']/-365).dropna(), vert=False)
plt.show()

Нарешті, тепер логічно замінити усі значення на додатні, щоб далі не перейматися щодо цього

In [ ]:
train['DAYS_BIRTH'] = abs(train['DAYS_BIRTH'])
test['DAYS_BIRTH'] = abs(test['DAYS_BIRTH'])
train['DAYS_EMPLOYED'] = abs(train['DAYS_EMPLOYED'])
test['DAYS_EMPLOYED'] = abs(test['DAYS_EMPLOYED'])

---

# Кореляції в даних

Тепер розрахуємо коефіцієнти кореляції колонки `TRAIN` з іншими в трейновому датасеті

In [ ]:
train['TARGET'] = train_labels
correlations = train.corr()['TARGET'].sort_values()

10 найбільших додатних коефіцієнтів кореляції

In [ ]:
correlations.tail(11).head(10)[::-1]

Тут найбільші коефіцієнти кореляції у `REGION_RATING_CLIENT_W_CITY` та `REGION_RATING_CLIENT`, що означають якимось чином оцінений рейтинг регіону/міста, де живе людина.

In [ ]:
print('REGION_RATING_CLIENT_W_CITY:',
      descr[descr['Row'] == 'REGION_RATING_CLIENT_W_CITY']['Description'].to_numpy()[0])
print('REGION_RATING_CLIENT:',
      descr[descr['Row'] == 'REGION_RATING_CLIENT']['Description'].to_numpy()[0])

10 найменших від'ємних коефіцієнтів кореляції

In [ ]:
correlations.head(10)

Бачимо, що показники `EXT_SOURCE`, які, згідно опису, означають Normalized score from external data source, мають великі від'ємні коефіцієнти кореляції. Після них ідуть `DAYS_BIRTH` та `DAYS_EMPLOYED`: зі збільшенням їх значення зменьшується шанс того, що проблеми з оплатою будуть.

Також, цікавою є ситуація з показниками `CODE_GENDER_F` та `CODE_GENDER_M`, що є кодуванням статі людини. Подивимося на розподіл `TARGET` в залежності від статі

In [ ]:
female_negative = sum(train[train['CODE_GENDER_F'] == 1]['TARGET'] == 0)
female_positive = sum(train[train['CODE_GENDER_F'] == 1]['TARGET'] == 1)
male_negative = sum(train[train['CODE_GENDER_M'] == 1]['TARGET'] == 0)
male_positive = sum(train[train['CODE_GENDER_M'] == 1]['TARGET'] == 1)
female_count = len(train[train['CODE_GENDER_F'] == 1])
male_count = len(train[train['CODE_GENDER_M'] == 1])
print("female:")
print(f"    don\'t have problems: {female_negative/female_count:.3f}%")
print(f"    have problems: {female_positive/female_count:.3f}%")
print("male:")
print(f"    don\'t have problems: {male_negative/male_count:.3f}%")
print(f"    have problems: {male_positive/male_count:.3f}%")

Бачимо, що серед жінок частка тих, у кого є проблеми з оплатою, трохи менша, ніж серед чоловіків

Тепер подивимося на розподіл віку в роках в залежності від наявності проблем з оплатою

In [ ]:
plt.boxplot(train[train['TARGET'] == 0]['DAYS_BIRTH']/365, vert=False)
plt.show()
print((train[train['TARGET'] == 0]['DAYS_BIRTH']/365).describe())
plt.boxplot(train[train['TARGET'] == 1]['DAYS_BIRTH']/365, vert=False)
plt.show()
print((train[train['TARGET'] == 1]['DAYS_BIRTH']/365).describe())

Отже, бачимо, що медіанний вік людей, у яких є проблеми з оплатою, менше за медіанний вік людей, у яких проблем немає. До того ж, видно, що проблеми з оплатою частіше виникають у більш молодих людей

Тепер зробимо теж саме з `DAYS_EMPLOYED`

In [ ]:
plt.boxplot(train[train['TARGET'] == 0]['DAYS_EMPLOYED'].dropna()/365, vert=False)
plt.show()
print((train[train['TARGET'] == 0]['DAYS_EMPLOYED'].dropna()/365).describe())
plt.boxplot(train[train['TARGET'] == 1]['DAYS_EMPLOYED'].dropna()/365, vert=False)
plt.show()
print((train[train['TARGET'] == 1]['DAYS_EMPLOYED'].dropna()/365).describe())

Для цього показника немає візуальної різниці між тими, хто має проблеми, і тими, хто немає.

Додамо ще один показник, рівний `DAYS_EMPLOYED/DAYS_BIRTH` - яку частину свого життя людина працює на поточній (на момент подачі заявки) роботі

In [ ]:
train['DAYS_EMPLOYED_FRAC'] = train['DAYS_EMPLOYED']/train['DAYS_BIRTH']
test['DAYS_EMPLOYED_FRAC'] = test['DAYS_EMPLOYED']/test['DAYS_BIRTH']

In [ ]:
plt.boxplot(train[train['TARGET'] == 0]['DAYS_EMPLOYED_FRAC'].dropna(), vert=False)
plt.show()
print(train[train['TARGET'] == 0]['DAYS_EMPLOYED_FRAC'].describe())
plt.boxplot(train[train['TARGET'] == 1]['DAYS_EMPLOYED_FRAC'].dropna(), vert=False)
plt.show()
print(train[train['TARGET'] == 1]['DAYS_EMPLOYED_FRAC'].describe())

Якщо не враховувати людей, що досить довго працюють на поточній роботі, видно, що ті люди, які мають проблеми з оплатою, на поточній роботі працюють меншу частину життя, ніж ті, які проблем не мають.

Подивимося тепер на таємничі показники `EXT_SOURCE_1`, `EXT_SOURCE_2`, `EXT_SOURCE_3`: наприклад, на їх коефіцієнти кореляції

In [ ]:
ext_data = train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'DAYS_EMPLOYED_FRAC']]
ext_data_corrs = ext_data.corr()
ext_data_corrs

Бачимо, що `DAYS_EMPLOYED_FRAC` та усі показники `EXT_SOURCE` від'ємно корелюють із `TARGET`, але додатно - з `DAYS_BIRTH`

---

# Нарешті, класифікація

Приберемо колонку `SK_ID_CURR`, оскільки, згідно опису, це просто ID of loan in our sample, а потім заповнимо пропущенні значення медіанниим

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
if 'TARGET' in train:
    train_labels = train['TARGET']
    train = train.drop(columns=['TARGET'])
    
train_modified = train.drop(columns=['SK_ID_CURR'])
feature_names = list(train_modified.columns)
test_id = test['SK_ID_CURR']
test_modified = test.drop(columns=['SK_ID_CURR'])

In [ ]:
imputer = SimpleImputer(strategy='median')
imputer.fit(train_modified)
train_modified = imputer.transform(train_modified)
test_modified = imputer.transform(test_modified)

Поділимо `train` та `train_lables` на власне train-частину та validation-частину

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(train_modified, train_labels, random_state=42)

Виконаємо grid search для знаходження оптимальної кількості дерев у лісі

In [ ]:
'''from sklearn.model_selection import GridSearchCV

params = {'n_estimators': [32, 64, 128, 256, 512]}
gs_clf = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs=-1), params)
gs_clf.fit(x_train, y_train)
print(gs_clf.best_params_)'''

Натренуємо `RandomForestClassifier` та перевіримо ROC-AUC на validation-датасеті

In [ ]:
clf = RandomForestClassifier(n_estimators=128, random_state=42, n_jobs=-1)
clf.fit(x_train, y_train)

In [ ]:
pred_train = clf.predict_proba(x_train)[:, 1]
pred_val = clf.predict_proba(x_val)[:, 1]
print('ROC-AUC train', roc_auc_score(y_train.values, pred_train))
print('ROC-AUC validation', roc_auc_score(y_val.values, pred_val))

Подивимося на feature importance, RandomForestClassifier може дати цю інформацію

In [ ]:
feature_importance = pd.DataFrame({'feature': feature_names,
                                   'importance': clf.feature_importances_})

In [ ]:
feature_importance.sort_values(by='importance', ascending=False).head(10)

Бачимо, що `EXT_SOURCE_2`, `EXT_SOURCE_3`, `DAYS_BIRTH`, `DAYS_EMPLOYED` та `DAYS_EMPLOYED_FRAC` дійсно мають великий вплив у моделі, як і передбачалося при дослідженні кореляції. Подивимося на ще два показники з великим значенням

In [ ]:
print('DAYS_ID_PUBLISH:',
      descr[descr['Row'] == 'DAYS_ID_PUBLISH']['Description'].to_numpy()[0])
print('DAYS_REGISTRATION:',
      descr[descr['Row'] == 'DAYS_REGISTRATION']['Description'].to_numpy()[0])

Залишилося зробити передбачення на тестових даних, яке потім завантажимо на Kaggle

In [ ]:
test_pred = clf.predict_proba(test_modified)[:, 1]
sub = pd.DataFrame({'SK_ID_CURR': test_id, 'TARGET': test_pred})
sub.to_csv('./my_submission.csv', index=False)